In [28]:
from datetime import datetime, timedelta
import pandas as pd
import plotly.figure_factory as ff
from ortools.sat.python import cp_model
from bisect import bisect_left, bisect_right
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px


In [29]:
from datetime import datetime, timedelta

from datetime import datetime, timedelta

class TimeInterval:
    def __init__(self, start_time, end_time):
        self.start_time = start_time
        self.end_time = end_time

    def __repr__(self):
        return f'TimeInterval({self.start_time}, {self.end_time})'

    def __str__(self):
        return f'{self.start_time} - {self.end_time}'

    def __eq__(self, other):
        return self.start_time == other.start_time and self.end_time == other.end_time

    def __lt__(self, other):
        return self.end_time <= other.start_time

    def __gt__(self, other):
        return self.start_time >= other.end_time

    def __add__(self, other):
        if self > other:
            return TimeIntervals([other, self])
        elif self < other:
            return TimeIntervals([self, other])
        else:
            return TimeIntervals([self])

    def __sub__(self, other):
        if self == other:
            return TimeIntervals([])
        elif self < other:
            return TimeIntervals([self])
        elif self > other:
            return TimeIntervals([TimeInterval(other.end_time, self.end_time)])

    def contains(self, other):
        return self.start_time <= other.start_time and self.end_time >= other.end_time


class TimeIntervals:
    def __init__(self, intervals):
        self.intervals = self.combine_intervals(intervals)

    def __repr__(self):
        return f'TimeIntervals({self.intervals})'

    def __str__(self):
        return ', '.join(str(interval) for interval in self.intervals)

    def __add__(self, other):
        intervals = self.intervals + other.intervals
        return TimeIntervals(intervals)

    # Complex version
    # def __sub__(self, other):
    #     result_intervals = []
    #     for interval in self.intervals:
    #         for sub_interval in other.intervals:
    #             if interval.start_time < sub_interval.start_time and interval.end_time > sub_interval.end_time:
    #                 result_intervals.append(TimeInterval(interval.start_time, sub_interval.start_time))
    #                 result_intervals.append(TimeInterval(sub_interval.end_time, interval.end_time))
    #             elif interval.start_time >= sub_interval.start_time and interval.end_time <= sub_interval.end_time:
    #                 pass
    #             elif interval.start_time >= sub_interval.start_time and interval.start_time < sub_interval.end_time and interval.end_time > sub_interval.end_time:
    #                 interval.start_time = sub_interval.end_time
    #             elif interval.start_time < sub_interval.start_time and interval.end_time > sub_interval.start_time and interval.end_time <= sub_interval.end_time:
    #                 interval.end_time = sub_interval.start_time
    #             else:
    #                 result_intervals.append(interval)
    #     return TimeIntervals(result_intervals)

    # More efficient version - using sweep line algorithm
    def __sub__(self, other):
        intervals = self.intervals + other.intervals
        events = []
        for interval in intervals:
            events.append((interval.start_time, 1))
            events.append((interval.end_time, -1))
        events.sort()
        result_intervals = []
        count = 0
        start_time = None
        for time, delta in events:
            if count == 0 and start_time is not None:
                result_intervals.append(TimeInterval(start_time, time))
            count += delta
            if count == 0:
                start_time = None
            else:
                start_time = time
        return TimeIntervals(result_intervals)


    def contains(self, interval):
        for iv in self.intervals:
            if iv.contains(interval):
                return True
        return False

    # @staticmethod
    # def combine_intervals(intervals):
    #     intervals.sort(key=lambda x: x.start_time)
    #     merged_intervals = []
    #     for interval in intervals:
    #         if not merged_intervals:
    #             merged_intervals.append(interval)
    #         else:
    #             last_interval = merged_intervals[-1]
    #             if last_interval.end_time >= interval.start_time:
    #                 merged_intervals[-1] = TimeInterval(last_interval.start_time, max(last_interval.end_time, interval.end_time))
    #             else:
    #                 merged_intervals.append(interval)
    #     return merged_intervals

    @staticmethod
    def combine_intervals(intervals):
        intervals.sort(key=lambda x: x.start_time)
        merged_intervals = []
        for interval in intervals:
            idx = bisect_right(merged_intervals, interval)
            if idx == 0 or interval.start_time > merged_intervals[idx-1].end_time:
                merged_intervals.insert(idx, interval)
            else:
                merged_intervals[idx-1].end_time = max(merged_intervals[idx-1].end_time, interval.end_time)
        return merged_intervals

    def visualize_gantt(self):
        # Combine intervals into a single interval
        combined_interval = self.combine_intervals(self)

        # Create a Pandas DataFrame with the interval data
        data = pd.DataFrame({'Task': ['Time Intervals'],
                            'Start': [combined_interval.start_time],
                            'Finish': [combined_interval.end_time]})

        # Create the Gantt chart
        fig = px.timeline(data, x_start='Start', x_end='Finish', y='Task', title='Time Intervals')

        # Format the chart
        fig.update_yaxes(autorange="reversed", showgrid=False, showticklabels=True)
        fig.update_xaxes(showgrid=True, showticklabels=True)
        fig.update_layout(height=300, margin=dict(l=0, r=0, t=30, b=0))

        # Show the chart
        fig.show()



In [30]:
intervals = TimeIntervals([TimeInterval(datetime(2023, 2, 1, 10), datetime(2023, 2, 1, 11)),
                           TimeInterval(datetime(2023, 2, 1, 12), datetime(2023, 2, 1, 13)),
                           TimeInterval(datetime(2023, 2, 1, 14), datetime(2023, 2, 1, 16))])
intervals.visualize_gantt()


AttributeError: 'TimeIntervals' object has no attribute 'sort'